In [1]:
import numpy as np
import lightgbm as lgb
from wideboost.wrappers import wlgb
from sklearn import datasets

In [2]:
digits = datasets.load_digits()

train_idx = np.random.choice(np.arange(1797),round(1797*0.7),replace=False)
test_idx = np.setdiff1d(np.arange(1797),train_idx)


xtrain = digits['data']
ytrain = digits['target']


train_data = lgb.Dataset(xtrain[train_idx,:], label=ytrain[train_idx])
test_data = lgb.Dataset(xtrain[test_idx,:], label=ytrain[test_idx])

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval

best_val = 1.0

def objective(param):
    global best_val
    #watchlist = [(dtrain,'train'),(dtest,'test')]
    ed1_results = dict()
    print(param)
    param['num_leaves'] = round(param['num_leaves']+1)
    param['min_data_in_leaf'] = round(param['min_data_in_leaf'])
    wbst = wlgb.train(param,
                      train_data,
                      num_boost_round=100,
                      valid_sets=test_data,
                      evals_result=ed1_results)
    output = min(ed1_results['valid_0']['multi_error'])
    
    if output < best_val:
        print("NEW BEST VALUE!")
        best_val = output
    
    return {'loss': output, 'status': STATUS_OK }

spc = {
    'btype': hp.choice('btype',['R','I','Rn','In']),
    'extra_dims': hp.choice('extra_dims',range(16)),
    'objective': hp.choice('objective',['multiclass']),
    'metric':hp.choice('metric',['multi_error']),
    'num_class':hp.choice('num_class',[10]),
    'learning_rate': hp.loguniform('learning_rate', -7, 0),
    'num_leaves' : hp.qloguniform('num_leaves', 0, 7, 1),
    'feature_fraction': hp.uniform('feature_fraction', 0.5, 1),
    'bagging_fraction': hp.uniform('bagging_fraction', 0.5, 1),
    'min_data_in_leaf': hp.qloguniform('min_data_in_leaf', 0, 6, 1),
    'min_sum_hessian_in_leaf': hp.loguniform('min_sum_hessian_in_leaf', -16, 5),
    'lambda_l1': hp.choice('lambda_l1', [0, hp.loguniform('lambda_l1_positive', -16, 2)]),
    'lambda_l2': hp.choice('lambda_l2', [0, hp.loguniform('lambda_l2_positive', -16, 2)])
}


best = fmin(objective,
    space=spc,
    algo=tpe.suggest,
    max_evals=100)

{'bagging_fraction': 0.9415659907118574, 'btype': 'R', 'extra_dims': 1, 'feature_fraction': 0.5258310392792555, 'lambda_l1': 0, 'lambda_l2': 0.6819264312496398, 'learning_rate': 0.15664209971374746, 'metric': 'multi_error', 'min_data_in_leaf': 15.0, 'min_sum_hessian_in_leaf': 6.657181326296676e-05, 'num_class': 10, 'num_leaves': 17.0, 'objective': 'multiclass'}
Overwriting param `num_class`                          
Overwriting param `objective` while setting `fobj` in train.
Moving param `metric` to an feval.                     
[1]	valid_0's multi_error: 0.159555                    
[2]	valid_0's multi_error: 0.113173                    
[3]	valid_0's multi_error: 0.0853432                   
[4]	valid_0's multi_error: 0.0760668                   
[5]	valid_0's multi_error: 0.0630798                   
[6]	valid_0's multi_error: 0.0556586                   
[7]	valid_0's multi_error: 0.0445269                   
[8]	valid_0's multi_error: 0.0371058                   
[9]	valid_0's m

[17]	valid_0's multi_error: 0.0333952                                              
[18]	valid_0's multi_error: 0.0352505                                              
[19]	valid_0's multi_error: 0.0315399                                              
[20]	valid_0's multi_error: 0.0315399                                              
[21]	valid_0's multi_error: 0.0315399                                              
[22]	valid_0's multi_error: 0.0315399                                              
[23]	valid_0's multi_error: 0.0315399                                              
[24]	valid_0's multi_error: 0.0315399                                              
[25]	valid_0's multi_error: 0.0315399                                              
[26]	valid_0's multi_error: 0.0315399                                              
[27]	valid_0's multi_error: 0.0315399                                              
[28]	valid_0's multi_error: 0.0315399                                       

[6]	valid_0's multi_error: 0.0927644                                               
[7]	valid_0's multi_error: 0.0909091                                               
[8]	valid_0's multi_error: 0.0890538                                               
[9]	valid_0's multi_error: 0.0871985                                               
[10]	valid_0's multi_error: 0.0797774                                              
[11]	valid_0's multi_error: 0.0816327                                              
[12]	valid_0's multi_error: 0.0760668                                              
[13]	valid_0's multi_error: 0.0742115                                              
[14]	valid_0's multi_error: 0.0742115                                              
[15]	valid_0's multi_error: 0.0723562                                              
[16]	valid_0's multi_error: 0.0705009                                              
[17]	valid_0's multi_error: 0.0686456                                       

Overwriting param `num_class`                                                      
Overwriting param `objective` while setting `fobj` in train.                       
Moving param `metric` to an feval.                                                 
[1]	valid_0's multi_error: 0.584416                                                
[2]	valid_0's multi_error: 0.556586                                                
[3]	valid_0's multi_error: 0.534323                                                
[4]	valid_0's multi_error: 0.623377                                                
[5]	valid_0's multi_error: 0.632653                                                
[6]	valid_0's multi_error: 0.506494                                                
[7]	valid_0's multi_error: 0.552876                                                
[8]	valid_0's multi_error: 0.508349                                                
[9]	valid_0's multi_error: 0.500928                                         

[95]	valid_0's multi_error: 0.274583                                               
[96]	valid_0's multi_error: 0.270872                                               
[97]	valid_0's multi_error: 0.269017                                               
[98]	valid_0's multi_error: 0.265306                                               
[99]	valid_0's multi_error: 0.267161                                               
[100]	valid_0's multi_error: 0.267161                                              
{'bagging_fraction': 0.792438183508655, 'btype': 'In', 'extra_dims': 10, 'feature_fraction': 0.5762431068099819, 'lambda_l1': 1.2484656825209474e-07, 'lambda_l2': 0.004337291680943171, 'learning_rate': 0.0010883191438244106, 'metric': 'multi_error', 'min_data_in_leaf': 49.0, 'min_sum_hessian_in_leaf': 0.006235243366227712, 'num_class': 10, 'num_leaves': 8.0, 'objective': 'multiclass'}
Overwriting param `num_class`                                                      
Overwriting param `objec

...

[57]	valid_0's multi_error: 0.025974                                                 
[58]	valid_0's multi_error: 0.0241187                                                
[59]	valid_0's multi_error: 0.0241187                                                
[60]	valid_0's multi_error: 0.0241187                                                
[61]	valid_0's multi_error: 0.0241187                                                
[62]	valid_0's multi_error: 0.0241187                                                
[63]	valid_0's multi_error: 0.0241187                                                
[64]	valid_0's multi_error: 0.0241187                                                
[65]	valid_0's multi_error: 0.0241187                                                
[66]	valid_0's multi_error: 0.0241187                                                
[67]	valid_0's multi_error: 0.0241187                                                
[68]	valid_0's multi_error: 0.0241187                 

[45]	valid_0's multi_error: 0.0185529                                                
[46]	valid_0's multi_error: 0.0185529                                                
[47]	valid_0's multi_error: 0.0185529                                                
[48]	valid_0's multi_error: 0.0185529                                                
[49]	valid_0's multi_error: 0.0185529                                                
[50]	valid_0's multi_error: 0.0185529                                                
[51]	valid_0's multi_error: 0.0185529                                                
[52]	valid_0's multi_error: 0.0185529                                                
[53]	valid_0's multi_error: 0.0185529                                                
[54]	valid_0's multi_error: 0.0185529                                                
[55]	valid_0's multi_error: 0.0222635                                                
[56]	valid_0's multi_error: 0.0204082                 

[32]	valid_0's multi_error: 0.0204082                                                
[33]	valid_0's multi_error: 0.0204082                                                
[34]	valid_0's multi_error: 0.0204082                                                
[35]	valid_0's multi_error: 0.0204082                                                
[36]	valid_0's multi_error: 0.0204082                                                
[37]	valid_0's multi_error: 0.0204082                                                
[38]	valid_0's multi_error: 0.0204082                                                
[39]	valid_0's multi_error: 0.0204082                                                
[40]	valid_0's multi_error: 0.0204082                                                
[41]	valid_0's multi_error: 0.0204082                                                
[42]	valid_0's multi_error: 0.0204082                                                
[43]	valid_0's multi_error: 0.0204082                 

[20]	valid_0's multi_error: 0.0371058                                                
[21]	valid_0's multi_error: 0.0333952                                                
[22]	valid_0's multi_error: 0.0333952                                                
[23]	valid_0's multi_error: 0.0296846                                                
[24]	valid_0's multi_error: 0.0278293                                                
[25]	valid_0's multi_error: 0.0241187                                                
[26]	valid_0's multi_error: 0.0222635                                                
[27]	valid_0's multi_error: 0.0222635                                                
[28]	valid_0's multi_error: 0.0222635                                                
[29]	valid_0's multi_error: 0.0222635                                                
[30]	valid_0's multi_error: 0.0222635                                                
[31]	valid_0's multi_error: 0.0222635                 

[8]	valid_0's multi_error: 0.0315399                                                 
[9]	valid_0's multi_error: 0.0315399                                                 
[10]	valid_0's multi_error: 0.0296846                                                
[11]	valid_0's multi_error: 0.0315399                                                
[12]	valid_0's multi_error: 0.0296846                                                
[13]	valid_0's multi_error: 0.0278293                                                
[14]	valid_0's multi_error: 0.0278293                                                
[15]	valid_0's multi_error: 0.025974                                                 
[16]	valid_0's multi_error: 0.0241187                                                
[17]	valid_0's multi_error: 0.025974                                                 
[18]	valid_0's multi_error: 0.0204082                                                
[19]	valid_0's multi_error: 0.0204082                 

In [4]:
print(best_val)
print(space_eval(spc, best))

0.0037105751391465678
{'bagging_fraction': 0.8307193756206007, 'btype': 'Rn', 'extra_dims': 14, 'feature_fraction': 0.6985128195384421, 'lambda_l1': 0, 'lambda_l2': 0.0004138188636846172, 'learning_rate': 0.002239588062322523, 'metric': 'multi_error', 'min_data_in_leaf': 1.0, 'min_sum_hessian_in_leaf': 0.0002270888310199123, 'num_class': 10, 'num_leaves': 566.0, 'objective': 'multiclass'}
